In [80]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [86]:
class TwitterClient(object):
    
    def __init__(self):
        
        f = open("Keys.txt","r")
        lst = []
        for key in f:
            lst.append(key[:-1])
        consumer_key = lst[0]
        consumer_secret = lst[1]
        access_token = lst[2]
        access_token_secret = lst[3]
        
        try: 
            # create OAuthHandler object.It authorizes the consumer_key and consumer_secret
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
    
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements.It simply breaks the complex statements to simple statements
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

In [87]:
def main():
    
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Elections', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} %".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:\n") 
    for i,tweet in enumerate(ptweets[:10]): 
        print(f"{i+1}) {tweet['text']}") 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:\n") 
    for i,tweet in enumerate(ntweets[:10]): 
        print(f"{i+1}) {tweet['text']}")

In [88]:
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 30.120481927710845 %
Negative tweets percentage: 19.27710843373494 %
Neutral tweets percentage: 50.602409638554214 %


Positive tweets:

1) US elections 2020: Anti-Muslim conspiracy theorist wins seat in Congress https://t.co/IoI4Byn3qb
2) RT @verretor: Americans: Why do foreigners care so much about our elections?
US military bases: https://t.co/jmVb4PREWO
3) RT @JulianCastro: Republicans have now lost the popular vote in 7 of the last 8 presidential elections.
4) RT @decoin_io: The US #Elections are on their way, would it affect the bitcoin price?
If you Can guess the exact price of #Bitcoin on Novem…
5) That’s right but BMD machines are manipulated just like Americans into thinking they live in a country with free an… https://t.co/5s1U26Ei0p
6) @Salon My guess is that our political system is ready to enact legislation to improve election integrity and securi… https://t.co/HOpBUOaMNv
7) RT @goldengateblond: one reason we're getting fair counts in Michigan,